#PPC

Este notebook mostra como o texto do PPC é extraído, tokenizado e indexado para uso no modelo RAG.

### Configurações iniciais e instalações de dependências

In [5]:
!pip install PyMuPDF nltk rank-bm25 transformers python-dotenv google-generativeai --quiet # ou openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 31.2 MB/s eta 0:00:00


In [9]:
import os
import nltk
import sys
import nltk
from nltk.tokenize import word_tokenize

Downloads do NLTK

In [11]:
try:
    nltk.data.find("tokenizers/punkt")
except LookupError:
    nltk.download("punkt", quiet=True)

Import de **app** - código com as funções de extração, e tokenização com o motor BM25.

In [12]:
from app import extrair_texto_pdf, preparar_paragrafos, tokenizar_paragrafos, configurar_bm25

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Como foi feito e commitado num repositório privado, não é possível clonar no colab. Então adicionei o arquivo .py na pasta de arquivos do ambiente colab.

### Extração do Texto

In [13]:
# O caminho '../data/PPC-LC-atualizao.pdf' está correto se o notebook está em 'notebooks/' ou '/content/PPC-LC-atualizao.pdf' se estiver no colab.
texto = extrair_texto_pdf("/content/PPC-LC-atualizao.pdf")
print(f"Primeiros 1000 caracteres do texto extraído:\n{texto[:1000]}\n")

Primeiros 1000 caracteres do texto extraído:
Universidade Federal Rural de Pernambuco – UFRPE
Pró-Reitoria de Ensino de Graduação - PREG
Departamento de Computação - DC
Licenciatura em Computação
Projeto Pedagógico do Curso
Março de 2023
Sumário
Dados de Identificação
4
Dados gerais
4
Base legal do curso
4
1. Apresentação
7
2. Histórico da UFRPE
7
2.1. Histórico do curso
9
2.2. Justificativa
10
3. Objetivos
11
3.1. Objetivo geral
12
3.2. Objetivos específicos
12
4. Perfil do profissional
12
5. Competências, atitudes e habilidades
13
6. Campo de atuação do profissional
13
7. Requisitos de ingresso
14
8. Organização curricular
16
8.1. Funcionamento do curso
18
8.2. Matriz curricular
18
8.2.1. Componentes curriculares obrigatórios
18
8.2.2. Componentes curriculares optativos
24
8.2.3. Síntese da carga horária total do curso
27
8.3. Representação gráfica da matriz curricular
27
8.4. Programas de disciplinas por componente curricular
29
8.4.1. Programa dos componentes curriculares obrigatór

### Divisão em parágrafos e tokenização

In [14]:
paragrafos = preparar_paragrafos(texto)
tokenizados = tokenizar_paragrafos(paragrafos)
if paragrafos:
    print(f"Primeiro parágrafo:\n{paragrafos[0]}\n")
    print(f"Primeiro parágrafo tokenizado:\n{tokenizados[0]}\n")
else:
    print("Nenhum parágrafo foi extraído. Verifique a função preparar_paragrafos.")

Primeiro parágrafo:
Universidade Federal Rural de Pernambuco – UFRPE
Pró-Reitoria de Ensino de Graduação - PREG
Departamento de Computação - DC
Licenciatura em Computação
Projeto Pedagógico do Curso
Março de 2023
Sumário
Dados de Identificação
4
Dados gerais
4
Base legal do curso
4
1. Apresentação
7
2. Histórico da UFRPE
7
2.1. Histórico do curso
9
2.2. Justificativa
10
3. Objetivos
11
3.1. Objetivo geral
12
3.2. Objetivos específicos
12
4. Perfil do profissional
12
5. Competências, atitudes e habilidades
13
6. Cam

Primeiro parágrafo tokenizado:
['universidade', 'federal', 'rural', 'de', 'pernambuco', '–', 'ufrpe', 'pró-reitoria', 'de', 'ensino', 'de', 'graduação', '-', 'preg', 'departamento', 'de', 'computação', '-', 'dc', 'licenciatura', 'em', 'computação', 'projeto', 'pedagógico', 'do', 'curso', 'março', 'de', '2023', 'sumário', 'dados', 'de', 'identificação', '4', 'dados', 'gerais', '4', 'base', 'legal', 'do', 'curso', '4', '1.', 'apresentação', '7', '2.', 'histórico', 'da', 'ufrp

Indexação com BM25

O BM25 não “guarda” documentos em forma de vetores densos nem faz embeddings; em vez disso, trabalha com uma indexação invertida.

1. Tokeniza cada documento, e para cada token cria uma lista com IDs dos docs onde ele aparece e sua contagem (TF), calculando e armazenando a frequencia do documento de cada termo para uso no IDF.

2. Para a consulta, ele tokeniza a consulta do usuário e retorna documentos onde aquele termo está mais presente e devolve o top k (definido no exemplo abaixo em 3).

In [16]:
bm25 = configurar_bm25(tokenizados)

In [20]:
consulta = "Quantas horas tem a carga horária de psicologia ii?"
consulta_tokenizada = word_tokenize(consulta.lower())

top_docs = bm25.get_top_n(consulta_tokenizada, paragrafos, n=3)

In [21]:
print(f"Documentos recuperados pelo BM25 (Top 3):\n")
for i, doc in enumerate(top_docs):
    print(f"--- Documento {i+1} ---\n{doc}\n")

Documentos recuperados pelo BM25 (Top 3):

--- Documento 1 ---
: Editora Ciência Moderna, 2010. 410 p.
5.
DEITEL, Harvey M. et. Al. Java como Programar. Bookman, 2005.
*Essa disciplina poderá ter até 4 encontros a distância, se aprovado em plano de ensino pelo colegiado.
Nome: Psicologia II
Código: 5319
Departamento: Departamento de Educação
Área: Psicologia
Carga horária total: 60 horas
Créditos: 4
Carga horária semanal: 4 horas (teóricas: 4; práticas: 0; EAD*: 0)
Pré-requisitos: 5317- Psicologia I
Co-requisitos: Nenhum
Objetivos:
Geral: identificar e anal

--- Documento 2 ---
rá dividido em 4 estágios (I, II, III
e IV), iniciando no 6o período, ou seja, na segunda metade do curso. A disciplina de Estágio
105
I tem como pré-requisito as disciplinas Metodologia do Ensino da Computação e Educação
Brasileira: legislação, organização e políticas.
As disciplinas de estágio da LC possuem carga horária total de 405 horas divididas
em 120 horas de atividades teóricas e 285 horas de atividades

O Documento 1 já possui a resposta que almejamos, mas sem passar pelo modelo de linguagem, acaba trazendo contexto desnecessário para a resposta.